In [1]:
import mpisppy.utils.sputils as sputils
import pandas as pd
from utils.functions import extract_tem_min, get_data_fromNSRDB, power_PV_calculation, calculate_WT_power
import numpy as np
from utils.model import create_model
import json
from datetime import datetime

[    0.00] Initializing mpi-sppy


## Dimensionar MR OnGrid con escenarios de irradiancia, velocidad del viento y disponibilidad

### Seleccionar ubicación

In [2]:
location = {"lat":6.189, "lon":-67.485, "name":"Puerto Carreño", "year_deterministic":2019}
#location = {"lat":12.583, "lon":-81.706, "name":"San Andrés"}

### Escenario determinista

In [3]:
df, info = get_data_fromNSRDB(location["lat"], location["lon"], location["year_deterministic"])
date_vec = np.vectorize(datetime)
df_index = date_vec(df.Year.values,df.Month.values,df.Day.values, df.Hour.values, df.Minute.values, tzinfo=None)
df.index = df_index
df

,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle
2019-01-01 00:00:00,2019,1,1,0,0,0,0,0,1.1,23.4,161.98
2019-01-01 01:00:00,2019,1,1,1,0,0,0,0,1.2,23.0,153.19
2019-01-01 02:00:00,2019,1,1,2,0,0,0,0,1.4,22.7,140.91
2019-01-01 03:00:00,2019,1,1,3,0,0,0,0,1.5,22.4,127.58
2019-01-01 04:00:00,2019,1,1,4,0,0,0,0,1.5,22.1,113.88
...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00,2019,12,31,19,0,0,0,0,1.2,26.4,112.35
2019-12-31 20:00:00,2019,12,31,20,0,0,0,0,1.2,25.7,126.07
2019-12-31 21:00:00,2019,12,31,21,0,0,0,0,1.1,25.1,139.46
2019-12-31 22:00:00,2019,12,31,22,0,0,0,0,1.0,24.5,151.92


### Parámetros del modelo

In [4]:
data_model = {}

data_model["lat"] = location["lat"]
data_model["lon"] = location["lon"]

data_model["interest"] = 0.1
data_model["lifeyears"] = 25

data_model["load"] = {}
data_model["load"]["len"] = 8760
data_model["load"]["value"] = pd.read_csv("CaseData.csv")["LOAD"].to_numpy()
data_model["load"]["reactive"] = False

data_model["pv_modules"] = {}
data_model["pv_modules"]["type"] = pd.read_excel("Catalogo.xlsx", sheet_name = "PVModules", index_col = 0)

Temp_min = extract_tem_min(data_model["lat"], data_model["lon"])
for k in data_model["pv_modules"]["type"].columns:
    data_model["pv_modules"]["type"].loc['Voc_max', k] = np.round(data_model["pv_modules"]["type"].loc['Voc_STC',k]*(1+(data_model["pv_modules"]["type"].loc['Tc_Voc',k]/100)*(Temp_min-25)),2)


data_model["batteries"] = {}
data_model["batteries"]["type"] = pd.read_excel("Catalogo.xlsx", sheet_name = "BattModules", index_col = 0)

data_model["inverters"] = {}
data_model["inverters"]["type"] = pd.read_excel("Catalogo.xlsx", sheet_name = "Hybrid OnGrid", index_col = 0)
data_model["inverters"]["flex"] = False

data_model["windgen"] = {}
data_model["windgen"]["active"] = True
data_model["windgen"]["type"] = pd.read_excel("Catalogo.xlsx", sheet_name = "WindTurbines", index_col = 0)


data_model["ENS"] = {}
data_model["ENS"]["active"] = True
data_model["ENS"]["type"] = "fixed"
data_model["ENS"]["value"] = 16

data_model["grid"] = {}
data_model["grid"]["active"] = True
data_model["grid"]["pmax_buy"] = 200
data_model["grid"]["pmax_sell"] = 20

data_model["grid"]["buy_price"] = {}
data_model["grid"]["buy_price"]["type"] = "variable"
data_model["grid"]["buy_price"]["value"] = pd.read_csv("CaseData.csv")["C_BUY_PC"].to_numpy()

data_model["grid"]["sell_price"] = {}
data_model["grid"]["sell_price"]["type"] = "variable"
data_model["grid"]["sell_price"]["value"] = pd.read_csv("CaseData.csv")["C_SELL_PC"].to_numpy()

#https://cdn.ade-power.com/assets/pdf/generators/cat/cat-de200gc-data-sheet.pdf
data_model["generator"] = {}
data_model["generator"]["active"] = True
data_model["generator"]["fuel_cost"] = 0.59
data_model["generator"]["gen_cost"] = 0
data_model["generator"]["pmax"] = 175
data_model["generator"]["fmin"] = 1.45
data_model["generator"]["fmax"] = 50.8 
data_model["generator"]["fm"] = 0.282
data_model["generator"]["gen_OM_cost"] = 3
data_model["generator"]["min_p_load"] = 20

data_model["generator"]["av"] = {}
data_model["generator"]["av"]["active"] = True
data_model["generator"]["av"]["value"] = pd.read_csv("CaseData.csv")["AV_DIESEL"].to_numpy()

data_model["area"] = {}
data_model["area"]["active"] = False

data_model["max_invest"] = {}
data_model["max_invest"]["active"] = False  

data_model["environment"] = {}
data_model["environment"]["active"] = True
data_model["environment"]["mu"] = 266.76
data_model["environment"]["Cbono"] = 4.23

In [5]:
data_model["pv_modules"]["type"]

,Risen-590W,EcoGreen-540W
ID,,
Tecn,Monocristalino,Monocristalino
C_inst,275.02,253.24
C_OM_y,5,5
A,2.821,2.5764
P_stc,590,540
ty,25,25
deg,0.65,0.65
Voc_STC,41.2,49.4
Isc_STC,18.21,13.81


In [6]:
data_model["batteries"]["type"]

,PylonTech-UP5000,BYD-22.1
ID,,
Tecn,Litio,Litio
C_inst,2215,15300
C_OM_y,20,50
Cap_nom,4.8,22.08
Cap_inf,0.24,4.4
P_des,2.4,16
P_ch,2.4,16
n,0.97,0.95
ty,10,13


In [7]:
data_model["inverters"]["type"]

,IS-15kW,HYD-20kTL
ID,,
C_inst,5180.00,6000.00
C_OM_y,150.00,160.00
P_max_pv,22.50,30.00
Num_mpp,2.00,2.00
Num_in_mpp,1.00,1.00
Vdc_max_in,900.00,1000.00
V_mpp_inf,400.00,180.00
V_mpp_sup,800.00,960.00
Idc_max_in,37.60,25.00


In [8]:
data_model["windgen"]["type"]

,FX-20kW,TUGE-10kW
ID,,
Tec,Horizontal,Horizontal
C_inst,31000,12000
P_nom,20,10
C_OM_y,700,300
A,16,9
v_st,3,3
v_max,30,30
ty,20,20


## Resolver modelo esocástico

### Importar escenarios creados

In [9]:
abrv_name = ''.join([i for i in location["name"] if i.isupper()])

GHI_scens = pd.read_csv(f'Escenarios/GHI_scens_{abrv_name}.csv')
wind_scens = pd.read_csv(f'Escenarios/wind_scens_{abrv_name}.csv')
av_scens = pd.read_csv(f'Escenarios/av_scens_{abrv_name}.csv')


with open(f'Escenarios/GHI_prob_{abrv_name}.json') as user_file:
    GHI_prob = json.loads(user_file.read())
with open(f'Escenarios/wind_prob_{abrv_name}.json') as user_file:
    wind_prob = json.loads(user_file.read())
with open(f'Escenarios/av_prob_{abrv_name}.json') as user_file:
    av_prob = json.loads(user_file.read())


### Declarar función scenario_creator

In [10]:
def scenario_creator(scenario_name):

    scens = scenario_name.split("_")

    df_scen = df.copy()

    df_scen["GHI"] = GHI_scens[scens[0]].to_numpy().astype(float)
    df_scen["Wind Speed"] = wind_scens[scens[1]].to_numpy().astype(float)

    if df_scen.isnull().values.any():
        raise ValueError("NaN detected")

    data_model["pv_modules"]["Pmpp"] = power_PV_calculation(df_scen, data_model["pv_modules"]["type"], 0, 10, data_model["lat"])   
    
    Profiles, Wind_generation = calculate_WT_power(df_scen, data_model["windgen"]["type"], 0.001, 20, info['Elevation'].iloc[0])               
    data_model["windgen"]["generation"] = Wind_generation 

    data_model["grid"]["av"] = {}
    data_model["grid"]["av"]["active"] = True
    data_model["grid"]["av"]["value"] = av_scens[scens[2]].to_numpy()

    
    model = create_model(data_model)
    sputils.attach_root_node(model, model.FirstStage, [model.Xpvs, model.Xpv, model.XBs, model.XB, model.Bxch, model.XCh, model.XT])
    model._mpisppy_probability = GHI_prob[scens[0]]*wind_prob[scens[1]]*av_prob[scens[2]]
    return model

### Escenarios

In [11]:
escenarios = [f'{key_irr}_{key_wind}_{key_av}' for key_irr in GHI_prob for key_wind in wind_prob for key_av in av_prob]
escenarios

['bad_bad_scen1',
 'bad_bad_scen2',
 'bad_mean_scen1',
 'bad_mean_scen2',
 'bad_good_scen1',
 'bad_good_scen2',
 'mean_bad_scen1',
 'mean_bad_scen2',
 'mean_mean_scen1',
 'mean_mean_scen2',
 'mean_good_scen1',
 'mean_good_scen2',
 'good_bad_scen1',
 'good_bad_scen2',
 'good_mean_scen1',
 'good_mean_scen2',
 'good_good_scen1',
 'good_good_scen2']

### Resolver modelo estocástico

In [12]:
from mpisppy.opt.lshaped import LShapedMethod

bounds = {name: -200000 for name in escenarios}
options = {
    "root_solver": "cplex",
    "sp_solver": "cplex",
    "sp_solver_options" : {"threads" : 12},
    "max_iter": 30,
    "valid_eta_lb": bounds
}

ls = LShapedMethod(options, escenarios, scenario_creator)
result = ls.lshaped_algorithm()


[   67.99] Initializing SPBase
Current Iteration: 1 Time Elapsed:    0.00 Current Objective: -Inf
Current Iteration: 2 Time Elapsed:  895.15 Time Spent on Last Master:    0.29 Time Spent Generating Last Cut Set:  894.86 Current Objective: -3600000.00
Current Iteration: 3 Time Elapsed: 2661.25 Time Spent on Last Master:    0.19 Time Spent Generating Last Cut Set: 1765.89 Current Objective: -93010.45
Current Iteration: 4 Time Elapsed: 3941.38 Time Spent on Last Master:    0.19 Time Spent Generating Last Cut Set: 1279.94 Current Objective: 61674.36
Current Iteration: 5 Time Elapsed: 5446.45 Time Spent on Last Master:    0.24 Time Spent Generating Last Cut Set: 1504.82 Current Objective: 455480.93
Current Iteration: 6 Time Elapsed: 7248.02 Time Spent on Last Master:    0.25 Time Spent Generating Last Cut Set: 1801.30 Current Objective: 847810.75
Current Iteration: 7 Time Elapsed: 8689.49 Time Spent on Last Master:    0.22 Time Spent Generating Last Cut Set: 1441.25 Current Objective: 97431

In [13]:
variables = ls.gather_var_values_to_rank0()
for ((scen_name, var_name), var_value) in variables.items():
    print(scen_name, var_name, var_value)

bad_bad_scen1 Xpvs[EcoGreen-540W,HYD-20kTL] 0.0
bad_bad_scen1 Xpvs[EcoGreen-540W,IS-15kW] 0.0
bad_bad_scen1 Xpvs[Risen-590W,HYD-20kTL] 22.0
bad_bad_scen1 Xpvs[Risen-590W,IS-15kW] 0.0
bad_bad_scen1 Xpv[EcoGreen-540W,HYD-20kTL] 0.0
bad_bad_scen1 Xpv[EcoGreen-540W,IS-15kW] 0.0
bad_bad_scen1 Xpv[Risen-590W,HYD-20kTL] 506.0
bad_bad_scen1 Xpv[Risen-590W,IS-15kW] 0.0
bad_bad_scen1 XBs['BYD-22.1',HYD-20kTL] 0.0
bad_bad_scen1 XBs['BYD-22.1',IS-15kW] 0.0
bad_bad_scen1 XBs[PylonTech-UP5000,HYD-20kTL] 2.0
bad_bad_scen1 XBs[PylonTech-UP5000,IS-15kW] 0.0
bad_bad_scen1 XB['BYD-22.1',HYD-20kTL] 0.0
bad_bad_scen1 XB['BYD-22.1',IS-15kW] 0.0
bad_bad_scen1 XB[PylonTech-UP5000,HYD-20kTL] 16.0
bad_bad_scen1 XB[PylonTech-UP5000,IS-15kW] 0.0
bad_bad_scen1 Bxch[EcoGreen-540W,'BYD-22.1',HYD-20kTL] 0.0
bad_bad_scen1 Bxch[EcoGreen-540W,'BYD-22.1',IS-15kW] 0.0
bad_bad_scen1 Bxch[EcoGreen-540W,PylonTech-UP5000,HYD-20kTL] 0.0
bad_bad_scen1 Bxch[EcoGreen-540W,PylonTech-UP5000,IS-15kW] 0.0
bad_bad_scen1 Bxch[Risen-590

In [14]:
from pyomo.environ import value

VPN_F = np.sum([round(1/np.power(1+data_model["interest"],i),3) for i in np.arange(1,data_model["lifeyears"]+1)])
Init_invest = value(ls.local_scenarios[ls.local_scenario_names[0]].FirstStage)
LCOE = 0
CPN = Init_invest

for i in ls.local_scenarios.keys():
    m = ls.local_scenarios[i]
    CPN += value(m.SecondStage)*m._mpisppy_probability
    LCOE += ((value(m.FirstStage) + value(m.SecondStage))/(sum(m.Carga[t] - m.ENS[t].value + sum(m.PpvG[tch,t].value for tch in m.CH) + m.PTG[t].value for t in m.T)*VPN_F))*m._mpisppy_probability


print(f'Inversión incial: {Init_invest}, CPN: {CPN}, LCOE: {LCOE}')

Inversión incial: 317184.1, CPN: 1062631.0839887816, LCOE: 0.14128995667326277


In [15]:
ls.write_tree_solution(f'{abrv_name}_est_res')

### Resolver modelo deterministico - Escenario 1 disponibilidad

In [16]:
abrv_name = ''.join([i for i in location["name"] if i.isupper()])
av_scens = pd.read_csv(f'Escenarios/av_scens_{abrv_name}.csv')

In [17]:
data_model["pv_modules"]["Pmpp"] = power_PV_calculation(df, data_model["pv_modules"]["type"], 0, 10, data_model["lat"])   

Profiles, Wind_generation = calculate_WT_power(df, data_model["windgen"]["type"], 0.001, 20, info['Elevation'].iloc[0])               
data_model["windgen"]["generation"] = Wind_generation 

data_model["grid"]["av"] = {}
data_model["grid"]["av"]["active"] = True
data_model["grid"]["av"]["value"] = av_scens["scen1"].to_numpy()

In [18]:
def scenario_creator(scenario_name):
    
    model = create_model(data_model)
    sputils.attach_root_node(model, model.FirstStage, [model.Xpvs, model.Xpv, model.XBs, model.XB, model.Bxch, model.XCh, model.XT])
    model._mpisppy_probability = 1
    return model

In [19]:
escenarios = ["main"]

In [20]:
from mpisppy.opt.lshaped import LShapedMethod

bounds = {name: -200000 for name in escenarios}
options = {
    "root_solver": "cplex",
    "sp_solver": "cplex",
    "sp_solver_options" : {"threads" : 12},
    "max_iter": 30,
    "valid_eta_lb": bounds
}

ls = LShapedMethod(options, escenarios, scenario_creator)
result = ls.lshaped_algorithm()

[29736.35] Initializing SPBase
Current Iteration: 1 Time Elapsed:    0.00 Current Objective: -Inf
Current Iteration: 2 Time Elapsed:   38.60 Time Spent on Last Master:    0.20 Time Spent Generating Last Cut Set:   38.40 Current Objective: -200000.00
Current Iteration: 3 Time Elapsed:  115.57 Time Spent on Last Master:    0.22 Time Spent Generating Last Cut Set:   76.76 Current Objective: -93810.00
Current Iteration: 4 Time Elapsed:  183.99 Time Spent on Last Master:    0.12 Time Spent Generating Last Cut Set:   68.30 Current Objective: -13691.27
Current Iteration: 5 Time Elapsed:  530.30 Time Spent on Last Master:    0.12 Time Spent Generating Last Cut Set:  346.19 Current Objective: 427275.00
Current Iteration: 6 Time Elapsed:  647.57 Time Spent on Last Master:    0.71 Time Spent Generating Last Cut Set:  116.53 Current Objective: 722609.71
Current Iteration: 7 Time Elapsed:  745.77 Time Spent on Last Master:    0.12 Time Spent Generating Last Cut Set:   98.07 Current Objective: 79719

In [21]:
variables = ls.gather_var_values_to_rank0()
for ((scen_name, var_name), var_value) in variables.items():
    print(scen_name, var_name, var_value)

main Xpvs[EcoGreen-540W,HYD-20kTL] 0.0
main Xpvs[EcoGreen-540W,IS-15kW] 0.0
main Xpvs[Risen-590W,HYD-20kTL] 22.0
main Xpvs[Risen-590W,IS-15kW] 0.0
main Xpv[EcoGreen-540W,HYD-20kTL] 0.0
main Xpv[EcoGreen-540W,IS-15kW] 0.0
main Xpv[Risen-590W,HYD-20kTL] 506.0
main Xpv[Risen-590W,IS-15kW] 0.0
main XBs['BYD-22.1',HYD-20kTL] 0.0
main XBs['BYD-22.1',IS-15kW] 0.0
main XBs[PylonTech-UP5000,HYD-20kTL] 0.0
main XBs[PylonTech-UP5000,IS-15kW] 0.0
main XB['BYD-22.1',HYD-20kTL] 0.0
main XB['BYD-22.1',IS-15kW] 0.0
main XB[PylonTech-UP5000,HYD-20kTL] 0.0
main XB[PylonTech-UP5000,IS-15kW] 0.0
main Bxch[EcoGreen-540W,'BYD-22.1',HYD-20kTL] 0.0
main Bxch[EcoGreen-540W,'BYD-22.1',IS-15kW] 0.0
main Bxch[EcoGreen-540W,PylonTech-UP5000,HYD-20kTL] 0.0
main Bxch[EcoGreen-540W,PylonTech-UP5000,IS-15kW] 0.0
main Bxch[Risen-590W,'BYD-22.1',HYD-20kTL] 1.0
main Bxch[Risen-590W,'BYD-22.1',IS-15kW] 0.0
main Bxch[Risen-590W,PylonTech-UP5000,HYD-20kTL] 0.0
main Bxch[Risen-590W,PylonTech-UP5000,IS-15kW] 0.0
main XCh[EcoG

In [22]:
from pyomo.environ import value

VPN_F = np.sum([round(1/np.power(1+data_model["interest"],i),3) for i in np.arange(1,data_model["lifeyears"]+1)])
Init_invest = value(ls.local_scenarios[ls.local_scenario_names[0]].FirstStage)
LCOE = 0
CPN = Init_invest

for i in ls.local_scenarios.keys():
    m = ls.local_scenarios[i]
    CPN += value(m.SecondStage)*m._mpisppy_probability
    LCOE += ((value(m.FirstStage) + value(m.SecondStage))/(sum(m.Carga[t] - m.ENS[t].value + sum(m.PpvG[tch,t].value for tch in m.CH) + m.PTG[t].value for t in m.T)*VPN_F))*m._mpisppy_probability


print(f'Inversión incial: {Init_invest}, CPN: {CPN}, LCOE: {LCOE}')

Inversión incial: 259878.74, CPN: 981392.3038241876, LCOE: 0.13012831732302385


In [23]:
ls.write_tree_solution(f'{abrv_name}_det_res_scen1')

## Resolver modelo deterministico - Escenario 2 de disponibilidad

In [24]:
data_model["pv_modules"]["Pmpp"] = power_PV_calculation(df, data_model["pv_modules"]["type"], 0, 10, data_model["lat"])   

Profiles, Wind_generation = calculate_WT_power(df, data_model["windgen"]["type"], 0.001, 20, info['Elevation'].iloc[0])               
data_model["windgen"]["generation"] = Wind_generation 

data_model["grid"]["av"] = {}
data_model["grid"]["av"]["active"] = True
data_model["grid"]["av"]["value"] = av_scens["scen2"].to_numpy()

In [25]:
def scenario_creator(scenario_name):
    
    model = create_model(data_model)
    sputils.attach_root_node(model, model.FirstStage, [model.Xpvs, model.Xpv, model.XBs, model.XB, model.Bxch, model.XCh, model.XT])
    model._mpisppy_probability = 1
    return model

In [26]:
escenarios = ["main"]

In [27]:
from mpisppy.opt.lshaped import LShapedMethod

bounds = {name: -200000 for name in escenarios}
options = {
    "root_solver": "cplex",
    "sp_solver": "cplex",
    "sp_solver_options" : {"threads" : 12},
    "max_iter": 30,
    "valid_eta_lb": bounds
}

ls = LShapedMethod(options, escenarios, scenario_creator)
result = ls.lshaped_algorithm()

[31636.47] Initializing SPBase
Current Iteration: 1 Time Elapsed:    0.00 Current Objective: -Inf
Current Iteration: 2 Time Elapsed:   41.89 Time Spent on Last Master:    0.12 Time Spent Generating Last Cut Set:   41.78 Current Objective: -200000.00
Current Iteration: 3 Time Elapsed:  117.19 Time Spent on Last Master:    0.11 Time Spent Generating Last Cut Set:   75.19 Current Objective: -92451.88
Current Iteration: 4 Time Elapsed:  185.92 Time Spent on Last Master:    0.11 Time Spent Generating Last Cut Set:   68.61 Current Objective: -12332.09
Current Iteration: 5 Time Elapsed:  238.93 Time Spent on Last Master:    0.10 Time Spent Generating Last Cut Set:   52.91 Current Objective: 428429.24
Current Iteration: 6 Time Elapsed:  291.63 Time Spent on Last Master:    0.13 Time Spent Generating Last Cut Set:   52.57 Current Objective: 563181.37
Current Iteration: 7 Time Elapsed:  353.59 Time Spent on Last Master:    0.12 Time Spent Generating Last Cut Set:   61.85 Current Objective: 81633

In [28]:
variables = ls.gather_var_values_to_rank0()
for ((scen_name, var_name), var_value) in variables.items():
    print(scen_name, var_name, var_value)

main Xpvs[EcoGreen-540W,HYD-20kTL] 0.0
main Xpvs[EcoGreen-540W,IS-15kW] 0.0
main Xpvs[Risen-590W,HYD-20kTL] 26.0
main Xpvs[Risen-590W,IS-15kW] 0.0
main Xpv[EcoGreen-540W,HYD-20kTL] 0.0
main Xpv[EcoGreen-540W,IS-15kW] 0.0
main Xpv[Risen-590W,HYD-20kTL] 598.0
main Xpv[Risen-590W,IS-15kW] 0.0
main XBs['BYD-22.1',HYD-20kTL] 0.0
main XBs['BYD-22.1',IS-15kW] 0.0
main XBs[PylonTech-UP5000,HYD-20kTL] 11.0
main XBs[PylonTech-UP5000,IS-15kW] 0.0
main XB['BYD-22.1',HYD-20kTL] 0.0
main XB['BYD-22.1',IS-15kW] 0.0
main XB[PylonTech-UP5000,HYD-20kTL] 88.0
main XB[PylonTech-UP5000,IS-15kW] 0.0
main Bxch[EcoGreen-540W,'BYD-22.1',HYD-20kTL] 0.0
main Bxch[EcoGreen-540W,'BYD-22.1',IS-15kW] 0.0
main Bxch[EcoGreen-540W,PylonTech-UP5000,HYD-20kTL] 0.0
main Bxch[EcoGreen-540W,PylonTech-UP5000,IS-15kW] 0.0
main Bxch[Risen-590W,'BYD-22.1',HYD-20kTL] 0.0
main Bxch[Risen-590W,'BYD-22.1',IS-15kW] 0.0
main Bxch[Risen-590W,PylonTech-UP5000,HYD-20kTL] 1.0
main Bxch[Risen-590W,PylonTech-UP5000,IS-15kW] 0.0
main XCh[Ec

In [29]:
from pyomo.environ import value

VPN_F = np.sum([round(1/np.power(1+data_model["interest"],i),3) for i in np.arange(1,data_model["lifeyears"]+1)])
Init_invest = value(ls.local_scenarios[ls.local_scenario_names[0]].FirstStage)
LCOE = 0
CPN = Init_invest

for i in ls.local_scenarios.keys():
    m = ls.local_scenarios[i]
    CPN += value(m.SecondStage)*m._mpisppy_probability
    LCOE += ((value(m.FirstStage) + value(m.SecondStage))/(sum(m.Carga[t] - m.ENS[t].value + sum(m.PpvG[tch,t].value for tch in m.CH) + m.PTG[t].value for t in m.T)*VPN_F))*m._mpisppy_probability


print(f'Inversión incial: {Init_invest}, CPN: {CPN}, LCOE: {LCOE}')

Inversión incial: 622308.8999999999, CPN: 1176956.5470598508, LCOE: 0.15629312830433667


In [30]:
ls.write_tree_solution(f'{abrv_name}_det_res_scen2')

In [72]:
from mpisppy.opt.ef import ExtensiveForm

options = {"solver": "cplex"}

ef = ExtensiveForm(options, escenarios, scenario_creator)

solver_options={'mip_tolerances_mipgap':1e-2}
results = ef.solve_extensive_form(solver_options=solver_options, tee=True)



[47876.20] Initializing SPBase

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 20.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2020.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\Nicolas\AppData\Local\Temp\tmp7dl71gr5.cplex.log' open.
CPLEX> New value for mixed integer optimality gap tolerance: 0.01
CPLEX> Problem 'C:\Users\Nicolas\AppData\Local\Temp\tmpw72c5kls.pyomo.lp' read.
Read time = 0.70 sec. (53.15 ticks)
CPLEX> Problem name         : C:\Users\Nicolas\AppData\Local\Temp\tmpw72c5kls.pyomo.lp
Objective sense      : Minimize
Variables            :  359195  [Nneg: 315361,  Binary: 43808,
                                 General Integer: 26]
Objective nonzeros   :  157698
Linear constraints   :  385474  [Less: 315052,  Greater

In [73]:
objval = ef.get_objective_value()
print(f"{objval:.1f}")

soln = ef.get_root_solution()
for (var_name, var_val) in soln.items():
    print(var_name, var_val)

1175016.3
Xpvs[EcoGreen-540W,HYD-20kTL] 0.0
Xpvs[EcoGreen-540W,IS-15kW] 0.0
Xpvs[Risen-590W,HYD-20kTL] 26.0
Xpvs[Risen-590W,IS-15kW] 0.0
Xpv[EcoGreen-540W,HYD-20kTL] 0.0
Xpv[EcoGreen-540W,IS-15kW] 0.0
Xpv[Risen-590W,HYD-20kTL] 598.0
Xpv[Risen-590W,IS-15kW] 0.0
XBs['BYD-22.1',HYD-20kTL] 0.0
XBs['BYD-22.1',IS-15kW] 0.0
XBs[PylonTech-UP5000,HYD-20kTL] 11.0
XBs[PylonTech-UP5000,IS-15kW] 0.0
XB['BYD-22.1',HYD-20kTL] 0.0
XB['BYD-22.1',IS-15kW] 0.0
XB[PylonTech-UP5000,HYD-20kTL] 88.0
XB[PylonTech-UP5000,IS-15kW] 0.0
Bxch[EcoGreen-540W,'BYD-22.1',HYD-20kTL] 0.0
Bxch[EcoGreen-540W,'BYD-22.1',IS-15kW] 0.0
Bxch[EcoGreen-540W,PylonTech-UP5000,HYD-20kTL] 0.0
Bxch[EcoGreen-540W,PylonTech-UP5000,IS-15kW] 0.0
Bxch[Risen-590W,'BYD-22.1',HYD-20kTL] 0.0
Bxch[Risen-590W,'BYD-22.1',IS-15kW] 0.0
Bxch[Risen-590W,PylonTech-UP5000,HYD-20kTL] 1.0
Bxch[Risen-590W,PylonTech-UP5000,IS-15kW] 0.0
XCh[EcoGreen-540W,'BYD-22.1',HYD-20kTL] 0.0
XCh[EcoGreen-540W,'BYD-22.1',IS-15kW] -0.0
XCh[EcoGreen-540W,PylonTech-UP5000